In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import fiona
import collections
import matplotlib.pyplot as plt
import pyproj
import rasterio
import rasterio.mask
from rasterio.plot import show
from rasterio import features
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date, timedelta
import geopandas
import os
from shapely import geometry
import numpy as np
import utilities
import pandas as pd
import shutil
import glob
pd.set_option('display.max_columns', None)
import time

In [3]:
# 18-21 Jul 2019
start_date = date(2020, 4, 1)
end_date = date(2021, 3, 25)

In [4]:
path = "/home/primity/terras/bulk/"
dest = "/home/primity/terras/bulk_crops/"

In [5]:
os.makedirs(dest, exist_ok=True)
footprint = geojson_to_wkt(read_geojson('quadrado-macao.geojson'))
for current_date in utilities.daterange(start_date, end_date -  timedelta(1)):
    time.sleep(5)
    os.makedirs(path, exist_ok=True)
    #get products list from this day
    api = SentinelAPI('fernandeslouro', 'copernicospw', 'https://scihub.copernicus.eu/dhus')
    products = api.query(footprint,
                         date=(current_date, current_date + timedelta(1)),
                         platformname='Sentinel-2',
                         cloudcoverpercentage=(0, 30))
    products_df = api.to_dataframe(products)
    
    if products:
        print(f"{current_date} - {len(products)} available products")
        # download them all
        for p in list(products_df.index):
            time.sleep(10)
            utilities.download_no_fail(p, 'fernandeslouro', 'copernicospw', 'https://scihub.copernicus.eu/dhus')
            print(p)
        # unzip them all
        for f in os.listdir(path):
            bashCmd = f"!unzip {os.path.join(path, f)}"
            process = subprocess.Popen(bashCmd, stdout=subprocess.PIPE)
            output, error = process.communicate()
        # copy jp2 to dest
        utilities.subfolders_copy(path, dest)
        
        # save cropped jp2 with same name 
        for i in os.listdir(dest):
            with rasterio.open(os.path.join(image_dir, i)) as src:
                out_image, out_transform = rasterio.mask.mask(src, utilities.transform_shapefile(macao_shp), crop=True, nodata=10, all_touched=True)
                out_meta = src.meta.copy() 
            with rasterio.open(os.path.join(image_dir, i), 'w', **profile) as dst:
                dst.write(out_image.astype(rasterio.uint8), 1)
                
        
        shutil.rmtree(path) 
    

2019-04-20 - 8 available products
Trying to download 437a64f6-f761-4a69-bdd6-7563593653af


Product 437a64f6-f761-4a69-bdd6-7563593653af is not online. Triggering retrieval from long term archive.


437a64f6-f761-4a69-bdd6-7563593653af
2019-04-30 - 8 available products
Trying to download f95060c0-9741-49cf-bba2-72bfc977f3e2


Product f95060c0-9741-49cf-bba2-72bfc977f3e2 is not online. Triggering retrieval from long term archive.


f95060c0-9741-49cf-bba2-72bfc977f3e2


KeyboardInterrupt: 